# Advent of Code 2016 Day 20
http://adventofcode.com/2016/day/20

In [1]:
import pandas as pd
import numpy as np

MAXINT = 4294967295

# Part 1 in `numpy` plus a Python function
First we load the data into a numpy array and sort it by row so that our blacklist ranges are in increasing order of start address

In [2]:
fwRules = np.loadtxt('input.txt', delimiter='-', dtype=np.uint32)
fwRules.sort(axis=0)

`first_gap` looks through the array for the first allowed IP.  I know that there is at least one such in the data, so I'm not testing for running off the end of the array.  Below,  I'll use pure pandas to get the first IP.

In [3]:
def first_gap(r):
    i = 1
    ls, le = r[0]
    while r[i][0] <= le + 1:
        if r[i][1] > le:
            le = r[i][1]
        i += 1
    return le + 1

first_gap(fwRules)

17348574

# Working in pure `pandas` 

## Part 1


In [4]:
fwRulesDF = pd.DataFrame(fwRules, columns=['start', 'end'])

To allow for the fact that the ends of ranges in the input might not be in monotonic order (we sorted first on the starts of the ranges), we add a column to keep track of the highest endpoint so far:-

In [5]:
fwRulesDF['maxEnd'] = fwRulesDF.cummax()['end']

Once we have the `maxEnd` column, the number of allowed IPs before each range in the table can be determined from the difference of the `start` on this row and the `maxEnd` on the previous row.  If this is greater than zero, it is the count of the allowed IPs. (if it's negative, it represents the size of overlap of this range with the ones prior to it).

In [6]:
fwRulesDF['ipcount'] = fwRulesDF['start'] - fwRulesDF['maxEnd'].shift(1) - 1
fwRulesDF.loc[fwRulesDF['ipcount'] < 0, 'ipcount'] = 0

Now, the first IP address in each block of allowed IPs can be calculated.  We select on column `ipcount` greater than 0 and subtract the number of allowed IPs from the start of the blacklist range to determine the first allowed IP in each window.

In [7]:
fwRulesDF['firstip'] = fwRulesDF[fwRulesDF['ipcount'] > 0]['start'] - fwRulesDF['ipcount']

fwRulesDF = fwRulesDF.fillna(0).astype(np.uint32)

The very first allowed IP is then the first non-zero entry in our `firstip` column:-

In [8]:
fwRulesDF[fwRulesDF['firstip'] > 0]['firstip'].iloc[0]

17348574

## Part 2
Part 2, the number of allowed IP addresses, comes easily.  We need to sum the `ipcount` column, then allow for the fact that the end of the last blacklist range might not be the upper limit of IP addresses (MAXINT).

In [9]:
np.sum(fwRulesDF['ipcount']) + (MAXINT - fwRulesDF['maxEnd'].iloc[-1])

104